# Dashboard on Hovering Data

In [1]:
import dash 
from dash import dcc, html, Input, Output, dash_table 
import plotly.express as px
import pandas as pd 
import json
import plotly.graph_objects as go 
import sys 
import base64

sys.path.append('../')

import omdutils as omd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
sql = 'SELECT * FROM wheels WHERE color in ("blue", "red", "yellow")'
db = '../SourceData/datasets.db'
df = omd.from_sqlite(sql, db) 
df

,wheels,color,image
0,1,red,redunicycle.jpg
1,1,blue,blueunicycle.jpg
2,1,yellow,yellowunicycle.jpg
3,2,red,redbicycle.jpg
4,2,blue,bluemotorcycle.jpg
5,2,yellow,yellowscooter.jpg
6,3,red,redtricycle.jpg
7,3,blue,bluetricycle.jpg
8,3,yellow,yellowrickshaw.jpg


In [4]:
app = dash.Dash(__name__)

def encode_image(image_file):
    try:
        encoded = base64.b64encode(open(image_file, 'rb').read())
        return 'data:image/png;base64,{}'.format(encoded.decode())
    except FileNotFoundError:
        return None
    
def get_marker_color(color):
    color_map = {
        'blue': 'rgb(70, 130, 180)',
        'yellow': 'rgb(255, 223, 0)',
        'red': 'rgb(255, 0, 0)'
    }
    return color_map.get(color, 'rgb(70, 130, 180)')  # Default color if not found


app.layout = html.Div([
    html.Div([
        dcc.Graph(
            id='wheels-plot',
            figure={
                'data': [
                    go.Scatter(
                        x=df['color'],
                        y=df['wheels'],
                        dy=1,
                        mode='markers',
                        marker={
                            'size': 12,
                            'color': [get_marker_color(color) for color in df['color']],
                            'line': {'width': 2}
                        }
                    )
                ],
                'layout': go.Layout(
                    title='Wheels & Colors Scatterplot',
                    xaxis={'title': 'Color'},
                    yaxis={'title': '# of Wheels', 'nticks': 3},
                    hovermode='closest'
                )
            }
        )
    ], style={'width': '30%', 'float': 'left'}),

    html.Div([
        html.Img(id='hover-image', src='children', height=300)
    ], style={'paddingTop': 35})
])

@app.callback(
    Output('hover-image', 'src'),
    [Input('wheels-plot', 'hoverData')]
)
def callback_image(hoverData):
    if hoverData is None:
        return ''
    
    try:
        wheel = hoverData['points'][0]['y']
        color = hoverData['points'][0]['x']
        path = '../Data/Images/'
        image_path = df[(df['wheels'] == wheel) & (df['color'] == color)]['image'].values[0]
        return encode_image(path + image_path)
    except (IndexError, KeyError):
        return ''


app.run_server(debug=True, port=8050)

nodename nor servname provided, or not known


AttributeError: 'tuple' object has no attribute 'tb_frame'